In [1]:
# 하이퍼 파라미터 튜닝
from ultralytics import YOLO
model = YOLO("yolo11x-obb.pt")
model.tune(data = "data2.yaml", epochs = 10, iterations=100, optimizer="AdamW", plots=False, save=True, val=True, lr0=1e-3)

Tuner: Initialized Tuner instance with 'tune_dir=runs\obb\tune7'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/100 with hyperparameters: {'lr0': 0.001, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
WARNING  training failure for hyperparameter tuning iteration 1
Command '['yolo', 'train', 'task=obb', 'mode=train', 'model=yolo11x-obb.pt', 'data=data2.yaml', 'epochs=10', 'time=None', 'patience=100', 'batch=16', 'imgsz=1024', 'save=True', 'save_period=-1', 'cache=False', 'device=None', 'workers=8', 'project=None', 'name=None', 'exist_ok=False', 'pretrained=True', 'optimizer=AdamW', 'verbose=True', 'seed=0', 'determinis

In [1]:
# 직접 YOLO 클래스를 임포트
from ultralytics import YOLO

# YOLO 모델 초기화
model = YOLO("yolo11x-obb.pt") 
results = model.train(
    data="data2.yaml",
    epochs=4000,
    batch=2,
    imgsz=640,
    patience=300,
    device="0",
    lr0 = 1e-3,
    
    weight_decay=0.005,
    flipud = 0.5,
    translate = 0.1,
    scale = 0.022,
    bgr = 0.2,
    mosaic = 0.0,
    
    copy_paste = 0.3,
    optimizer="AdamW",
    dropout = 0.3,
    augment = True,
    )

New https://pypi.org/project/ultralytics/8.3.9 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.5  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=obb, mode=train, model=yolo11x-obb.pt, data=data2.yaml, epochs=4000, time=None, patience=300, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train211, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

train: Scanning E:\Workspace\SatellieteIMGDetection\dataset2\labels\train.cache... 401 images, 0 backgrounds, 0 corrupt: 100%|██████████| 401/401 [00:00<?, ?it/s]
val: Scanning E:\Workspace\SatellieteIMGDetection\dataset2\labels\val.cache... 55 images, 0 backgrounds, 0 corrupt: 100%|██████████| 55/55 [00:00<?, ?it/s]


Plotting labels to runs\obb\train211\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 173 weight(decay=0.0), 183 weight(decay=0.005), 182 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\obb\train211
Starting training for 4000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     1/4000      4.37G       1.78      3.352      1.393         66        640: 100%|██████████| 201/201 [00:27<00:00,  7.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:03<00:00,  8.52it/s]

                   all         55       3257      0.237      0.309       0.18      0.111


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


   438/4000      4.21G     0.8487      0.684      1.069          3        640: 100%|██████████| 201/201 [00:24<00:00,  8.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:01<00:00, 18.66it/s]

                   all         55       3257      0.571      0.476      0.492      0.367



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


   439/4000       3.6G      0.864     0.6556      1.071        203        640:  33%|███▎      | 67/201 [00:08<00:16,  8.14it/s]

In [32]:
# YOLO 모델의 Detect 모듈을 제외한 백본만 추출
backbone = nn.Sequential(*list(yolo.model.model.children())[:-1])
backbone

Sequential(
  (0): Conv(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (1): Conv(
    (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (2): C3k2(
    (cv1): Conv(
      (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (cv2): Conv(
      (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (m): ModuleList(
      (0): C3k(
        (cv1): Conv(
      

In [34]:
head = nn.Sequential(list(yolo.model.model.children())[-1])
head

Sequential(
  (0): Detect(
    (cv2): ModuleList(
      (0): Sequential(
        (0): Conv(
          (conv): Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (1-2): 2 x Sequential(
        (0): Conv(
          (conv): Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(64, 64,

In [1]:
import torch
import torch.nn as nn
from ultralytics import YOLO

# YOLO 모델 불러오기 (YOLOv5 예시)
class YOLOv5WithCustomHead(nn.Module):
    def __init__(self, yolo):
        super(YOLOv5WithCustomHead, self).__init__()
        self.yolo = yolo.model  # YOLO 모델의 모든 레이어
        
        # YOLO의 중간층에서 사용할 레이어 인덱스를 확인해야 함
        self.custom_head = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),  # Global Average Pooling
            nn.Flatten(),
            nn.Linear(128, 10)  # 예시로 10개 클래스를 분류하는 헤드
        )

    def forward(self, x):
        outputs = []  # 여러 텐서를 저장할 리스트
        
        # 중간 피처 맵을 추출하는 부분
        for i, layer in enumerate(self.yolo.model):
            x = layer(x)
            if isinstance(x, list):  # x가 리스트인 경우
                x = torch.cat(x, dim=1)  # 리스트의 텐서를 결합
            if i == 9:  # 예시로 9번째 레이어 출력 사용
                feature_map = x  # 중간 피처 맵 추출
                outputs.append(feature_map)  # 필요한 피처 맵 저장

        # 기존 YOLO 아웃풋 (객체 탐지)
        yolo_output = x

        # 새로운 헤드에서 중간 피처 맵 사용
        custom_output = self.custom_head(feature_map)
        
        return yolo_output, custom_output

# 기존 YOLOv5 모델 불러오기
yolo = YOLO('yolo11m.pt')

# 커스텀 모델 생성 (YOLOv5에 새로운 헤드 추가)
model_with_custom_head = YOLOv5WithCustomHead(yolo)

# 예시 입력 데이터
input_data = torch.randn(1, 3, 640, 640)

# 모델 실행
yolo_output, custom_output = model_with_custom_head(input_data)

# 결과 출력
print("YOLO Output Shape:", yolo_output.shape)
print("Custom Head Output Shape:", custom_output.shape)



TypeError: cat() received an invalid combination of arguments - got (Tensor, int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [9]:
yolo.model

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
   